# Análisis de actividad sísmica en México

![Alerta](im.png)

Para los que tengan curiosidad de ver cómo se sacó la información de la página del SSN, hay otro .ipynb en este folder donde se explica detalladamente. Lo primero que hay que hacer es cargar los paquetes necesarios. Esto significa importar (e instalar anteriormente) los siguientes paquetes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

from helpers.Mexico import Republica

republica = Republica()

sns.set_style("darkgrid")

ModuleNotFoundError: No module named 'folium'

El paquete de seaborn lo pueden instalar con pip3, al igual que folium. helpers.Mexico es un paquete que yo hice para ayudarles un poco, de ahí saco la clase `Republica`.

De los paquetes necesarios, `Folium` es por mucho el más complejo. Permite hacer mapas interactivos en jupyter. Yo me he dado a la tarea de bajar del inegi las divisiones políticas del país y pasarlas a un formato que `Folium` entiende, conocido como topojson. El archivo se encuentra en el folder `helpers`. No es necesario que entiendan su contenido, pero yo les explicaré más adelante su uso.

Carguemos rápidamente el csv de la actividad sísmica reportada por el SSN de la UNAM desde el 1 de Enero de 1990 hasta el 17 de Febrero del 2018. Además de cargar el csv, vamos a ver los primeros 5 renglones, para familiarizarnos con el formato:

In [ ]:
#Definimos una lambda que convierta una cadena de caracteres en una etiqueta de tiempo.
mConverters = {'Fecha y Hora': lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') }
#Usamos la lambda para cargar el csv. Los nombres de las columnas son el primer renglón
sismos = pd.read_csv('helpers/earthquakes.csv', index_col = 0, converters = mConverters)
#Mostremos los primeros 5 renglones con la función head
sismos.head()

El csv es muy, muy sencillo. Contiene sólo 5 columnas: Magnitud, Latitud, Longitud, Profundidad y Fecha-Hora. Las primeras tres columnas son flotantes, mientras la quinta es una estampa de tiempo. Tendrán que aprender a usar estampas de tiempo pero yo les puedo ayudar.

## Ejercicio 1: Magnitud de los sismos

Calculen las siguientes cantidades (explicando en su código cómo lo hacen):
1. Cuenta total de sismos
2. Rango de magnitud de los sismos
3. Número de sismos por cada categoría, siendo las categorías {[1,2), [2,3),[3,4),[4,5),[5,6),[6,7),[7,8),[8,9)}
3. Número de sismos en cada año, graficado como un histograma (¿Ha aumentado la actividad sísmica?)

In [ ]:
'Cuenta total de sismos: {}'.format(len(sismos))

In [ ]:
# Rango de magnitud de sismos:

'El rango de los sismos registrados entre 1990 a 2018 es [{}, {}]'.format(sismos['Magnitud'].min(),
                                                                          sismos['Magnitud'].max())

In [ ]:
for i in range(1,9):
    res = sismos[(sismos.Magnitud >=i) & (sismos.Magnitud < (i + 1))].count()
    print('Número de sismos en rango [{0}, {1}): {2}'.format(i, i+1, res.Magnitud))

In [ ]:
% matplotlib notebook

conteo_año = sismos.groupby(sismos['Fecha y Hora'].dt.year).count()
conteo_año = conteo_año['Magnitud']

fig = plt.figure(figsize=(6, 4))
bars = plt.bar(conteo_año.index, conteo_año);
ax = fig.gca();
ax.set_ylabel('Conteo');
ax.set_xlabel('Año');
ax.set_title('Conteo de sismos detectados por año en México');

## Ejercicio 2: Graficando en el mapa

Para lo que viene, ustedes deben añadir una columna de nombre "Estado" al DataFrame `sismos`. Para ello, la variable `republica` declarada al principio del documento tiene una función llamada `estadoSegunPunto` que recibe como parámetros un par de números (latitud y longitud) y dice en qué estado de la república ocurrió el sismo (por ejemplo `republica.estadoSegunPunto(19,-100)`. La función devuelve `np.nan` si el punto no se encuentra dentro de un estado. Como ejercicio:

1. Calculen el porcentaje de epicentros dentro del territorio mexicano (en este caso el mar no cuenta).
2. Calculen cuántos sismos ocurrieron por estado.
3. Determinen cual es el estado con menor actividad sísmica (en cuanto a epicentros) y cuál el de mayor
4. Hagan un mapa de colores de la actividad sísmica por estado. Agarren buenos colores y usen una buena escala.

Para el cuarto punto, utilicen `Folium`. Les dejo el tip: las funciones que necesitan son `folium.Map()` (que genera un objeto, llamémoslo `mexmap`) y la función `.choropleth` (miembro de `mexmap`). Hay ejemplos en internet de esto, pero si tienen dudas y están muy muy frustrados, mándenme un correo y les ayudo un poco. Para generar el _choropleth_, necesitan el archivo `helpers/mexican_states.geojson`, la función `choropleth` se los va a pedir pero no tienen que entender lo que hay dentro. Ah, eso sí, los nombres de los estados están etiquetados por el nombre "NOM_ENT" (`choropleth` se los va a pedir...).

In [ ]:
sismos['Lugar'] = sismos.apply(lambda x: republica.estadoSegunPunto(x['Latitud'], x['Longitud']), axis=1)

In [ ]:
'Porcentaje de sismos en el territorio nacional: {}%'.format(sismos['Lugar'].count()*100/len(sismos))

In [ ]:
print('El número de sismos ocurridos en cada estado es:')
conteo_estados = sismos['Lugar'].value_counts();
conteo_estados = conteo_estados.to_frame().reset_index()
conteo_estados = conteo_estados.rename(index=str, columns={"index": "Estado", "Lugar": "Conteo"})
conteo_estados

In [ ]:
print('El estado con mayor número de epicentros contados es: {}'.format(sismos['Lugar'].value_counts().idxmax()))
print('El estado con menor número de epicentros contados es: {}'.format(sismos['Lugar'].value_counts().idxmin()))

In [ ]:
m = folium.Map([24,-100], zoom_start=5)

m.choropleth(
    geo_data='helpers/mexican_states.geojson',
    data=conteo_estados,
    columns=['Estado', 'Conteo'],
    key_on='feature.properties.NOM_ENT',
    fill_color='YlOrRd')
m

## Ejercicio 3: Terremotos

En este bloque:
1. Calculen cuántos sismos de magnitud igual o mayor a 6.8 han ocurrido en los últimos 28 años
2. Saquen la distancia temporal entre ellos, calculen su media y la desviación estándar (¿Algún patrón?)
3. Verifiquen, con el coeficiente de correlación, si hay dependencia entre magnitud y profundidad de los terremotos
4. **Colóquenlos en el mapa, de manera proporcional a la energía que liberaron** (un poco de ayuda después)

La tarea de graficar esto no es tan sencilla, por fortuna seguimos utilizando `folium`.

Para el 4to punto, necesito que hagan algo muy específico: La escala de Richter es una escala logaritmica de base 10, donde la magnitud $M_L$ se determina como.

$M_L = log_{10}(\frac{A}{A(\delta})$

siendo $A$ la amplitud registrada por un sismografo, y $A(\delta)$ una función de amplitud de referencia con una compensación basada en la distanca del sismógrafo al epicentro del sismo $\delta$. La relación de energía con la amplitud de una onda sismicaa es $E \propto A^{3/2}$. Esto quiere decir que un sismo de 5 grados libera $\approx 31.6$ veces menos energía que un sismo de 6 grados. Para graficar la localización y cantidad de energía liberada por estos sismos, hagan uso de la función "CircleMarker" de folium, y hagan una selección **adecuada** (en verdad enfatizo eso) del radio de los marcadores, sabiendo que lo que se busca es que los círculos representen la cantidad de energía liberada.

Además les dejo una función que hace un gradiente lineal de color entre dos tripletes RGB (red green blue) entre dos escalas de sismos (por defecto, 6.8 un extremo y 8.2 el otro). La pueden utilizar para darle color a los círculos también (pueden modificar la función para usar los colores que quieran)

In [ ]:
def intensityColor(intensity, minVal = 6.8, maxVal = 8.2):
    minimumColor, maximumColor = np.array([32, 193, 116]), np.array([243, 40, 40]) #par de tripletes RGB
    #Garantiza que el número está en el rango
    intensity = maxVal if intensity > maxVal else intensity
    intensity = minVal if intensity < minVal else intensity
    #Parametriza al intervalo 0,1
    intensity = (intensity - minVal) / (maxVal-minVal)
    #Asigna color linealmente
    currentColor = np.floor(maximumColor*(intensity) + minimumColor * (1 - intensity)).astype(int)
    #Regresa el color en formato hexadecimal, el cual CircleMarker entiende
    return '#%02x%02x%02x' % tuple(currentColor)

In [138]:
'El número de sismos con intensidad mayor a 6.8 es: {}'.format(sismos[sismos.Magnitud > 6.8].count()['Magnitud'])

'El número de sismos con intensidad mayor a 6.8 es: 20'

In [139]:
sismos_fuertes = sismos[sismos.Magnitud > 6.8]
print('La distancia temporal entre sismos de magnitud mayor a 6.8 es:')
sismos_fuertes['Fecha y Hora'].iloc[::-1].diff(periods=1)

La distancia temporal entre sismos de magnitud mayor a 6.8 es:


103275                  NaT
101987    733 days 18:51:36
101838     25 days 01:31:21
101763     11 days 11:03:04
101525    127 days 00:29:21
100789    321 days 17:20:07
100484    109 days 16:09:10
98310     775 days 09:04:28
97990     106 days 19:49:09
97008     313 days 19:10:34
93522     895 days 13:24:47
81966    2629 days 21:34:08
73928     715 days 18:22:06
70545     231 days 22:33:02
62645     526 days 22:51:31
60970      79 days 20:56:38
23934    1072 days 20:05:04
19741      85 days 21:20:15
16699      11 days 13:25:22
524       150 days 04:24:58
Name: Fecha y Hora, dtype: timedelta64[ns]

In [140]:
print('Media temporal de la separación: {}'.format(sismos_fuertes['Fecha y Hora'].iloc[::-1].diff(periods=1).mean()))
print('Desviación estándar temporal de la separación: {}'.format(sismos_fuertes['Fecha y Hora'].iloc[::-1].diff(periods=1).std()))
print('Media magnitud de sismos fuertes: {}'.format(sismos_fuertes['Magnitud'].mean()))
print('Desviación estándar de sismos fuertes: {}'.format(sismos_fuertes['Magnitud'].std()))

Media temporal de la separación: 469 days 17:55:05.315789
Desviación estándar temporal de la separación: 620 days 21:16:23.997631
Media magnitud de sismos fuertes: 7.264999999999999
Desviación estándar de sismos fuertes: 0.3407036063331601


In [141]:
print('La correlación para sismos fuertes entre profundidad y magnitud es: {}'.format(sismos_fuertes['Profundidad (km)'].corr(sismos_fuertes['Magnitud'])))
print('La correlación para todos los sismos entre profundidad y magnitud es: {}'.format(sismos['Profundidad (km)'].corr(sismos['Magnitud'])))

La correlación para sismos fuertes entre profundidad y magnitud es: -0.16194437208911105
La correlación para todos los sismos entre profundidad y magnitud es: 0.12929343367499696


## Ejercicio 4: Buscando patrones antes de un evento de gran escala (medio difícil)

En la siguiente parte, vamos a ver si, de manera sencilla, podemos encontrar patrones de comportamiento sísmico antes de un evento de gran escala (Terremoto de magnitud >= 7.0). Como primer ejercicio, encuentre cuántos sismos hay con dicha magnitud:

1. Definan un intervalo de tiempo a analizar antes de un sismo de gran magnitud, por ejemplo, 24 horas ó 5 días (justifiquen su escala de tiempo)
2. Para todos los sismos de magnitud igual o mayor a 7.0, grafiquen el comportamiento de la región (en un ángulo sólido prudente) en el periodo de tiempo que escogieron antes del sismo. Busquen si hay algún patrón evidente y concluyan.

Como tip, la diferencia de dos estampas de tiempo genera un pd.Timedelta. La siguiente función puede tomar un arreglo ( o una serie) de `pd.Timedelta` para convertirlo en horas:

In [ ]:
toHours = np.vectorize(lambda x : x.total_seconds()/3600)

Les va a ser mucho más fácil si trabajan en horas para este ejercicio.

In [ ]:
#Respuestas a partir de aquí

# Ejercicio 5: Qué más nos hace falta decir? (extra)

En esta última parte, pueden hacer **lo que quieran** con los datos. Agruparlos, clasificarlos, tienen libertad de sacar conclusiones de los datos que les he dado, pueden hacer algo con los mapas o hacer histogramas, queda en ustedes :)

In [ ]:
#Código aquí (si así lo quieren)